In [163]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import math
from sklearn.cross_validation import KFold
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.model_selection import RandomizedSearchCV,train_test_split,GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from xgboost.sklearn import XGBClassifier

In [164]:
datafile_train=r'C:\Users\sujith\Desktop\Project\p3\counterfeit_train.csv'
datafile_test=r'C:\Users\sujith\Desktop\Project\p3\counterfeit_test.csv'
cf_train=pd.read_csv(datafile_train)
cf_test=pd.read_csv(datafile_test)

In [165]:
cf_train.shape

(6818, 12)

In [166]:
cf_train.head(10)

,Medicine_ID,Counterfeit_Weight,DistArea_ID,Active_Since,Medicine_MRP,Medicine_Type,SidEffect_Level,Availability_rating,Area_Type,Area_City_Type,Area_dist_level,Counterfeit_Sales
0,RRA15,13.100,Area046,1995,160.2366,Antimalarial,critical,0.070422,DownTown,Tier 1,Small,1775.5026
1,YVV26,NaN,Area027,1983,110.4384,Mstablizers,mild,0.013000,CityLimits,Tier 3,Medium,3069.1520
2,LJC15,9.025,Area046,1995,259.4092,Cardiac,mild,0.060783,DownTown,Tier 1,Small,2603.0920
3,GWC40,11.800,Area046,1995,99.9830,OralContraceptives,mild,0.065555,DownTown,Tier 1,Small,1101.7130
4,QMN13,NaN,Area019,1983,56.4402,Hreplacements,critical,0.248859,MidTownResidential,Tier 1,Small,158.9402
5,JDG81,8.775,Area045,2000,165.5656,Antiseptics,mild,0.088881,DownTown,Tier 2,Unknown,3047.8464
6,KPX48,18.000,Area018,2007,156.9102,Hreplacements,critical,0.051685,Industrial,Tier 3,Medium,2883.3938
7,CYW14,7.680,Area013,1985,154.3470,Antiseptics,mild,0.028150,DownTown,Tier 3,High,3262.2340
8,TVF57,18.550,Area046,1995,107.3068,Hreplacements,critical,0.060266,DownTown,Tier 1,Small,1959.9292
9,MGU75,9.510,Area013,1985,161.1392,Statins,mild,0.023755,DownTown,Tier 3,High,3095.7840


In [167]:
cf_train.dtypes

Medicine_ID             object
Counterfeit_Weight     float64
DistArea_ID             object
Active_Since             int64
Medicine_MRP           float64
Medicine_Type           object
SidEffect_Level         object
Availability_rating    float64
Area_Type               object
Area_City_Type          object
Area_dist_level         object
Counterfeit_Sales      float64
dtype: object

In [168]:
for col in cf_train.select_dtypes(['object']).columns:
    print(col,':',cf_train[col].nunique())

Medicine_ID : 1557
DistArea_ID : 10
Medicine_Type : 16
SidEffect_Level : 2
Area_Type : 4
Area_City_Type : 3
Area_dist_level : 4


In [169]:
cf_train.isnull().sum()

Medicine_ID               0
Counterfeit_Weight     1166
DistArea_ID               0
Active_Since              0
Medicine_MRP              0
Medicine_Type             0
SidEffect_Level           0
Availability_rating       0
Area_Type                 0
Area_City_Type            0
Area_dist_level           0
Counterfeit_Sales         0
dtype: int64

In [170]:
cf_train.describe()

,Counterfeit_Weight,Active_Since,Medicine_MRP,Availability_rating,Counterfeit_Sales
count,5652.000000,6818.000000,6818.000000,6818.000000,6818.000000
mean,14.115057,1995.836316,151.401518,0.079174,2280.583480
std,4.649668,8.368979,62.203961,0.051481,1693.354404
min,5.855000,1983.000000,41.790000,0.013000,146.290000
25%,9.995000,1985.000000,104.509400,0.040058,933.265600
50%,13.800000,1997.000000,153.195700,0.066955,1902.670400
75%,18.050000,2002.000000,196.148350,0.107697,3207.638400
max,22.650000,2007.000000,277.188400,0.341391,13199.964800


In [171]:
cf_test.describe()

,Counterfeit_Weight,Active_Since,Medicine_MRP,Availability_rating
count,1408.000000,1705.000000,1705.000000,1705.000000
mean,14.328604,1995.814076,150.857964,0.078963
std,4.616129,8.385310,62.575018,0.052076
min,5.855000,1983.000000,41.590000,0.013000
25%,10.300000,1985.000000,102.880400,0.039831
50%,14.300000,1997.000000,153.715400,0.066693
75%,18.300000,2002.000000,195.389800,0.107220
max,22.650000,2007.000000,277.188400,0.324090


In [172]:
del cf_train['DistArea_ID']
del cf_test['DistArea_ID']
del cf_train['Medicine_ID']
del cf_test['Medicine_ID']

In [173]:
cf_train["Counterfeit_Weight"]=np.where(pd.isnull(cf_train["Counterfeit_Weight"]),14,cf_train['Counterfeit_Weight'])

In [174]:
cf_test["Counterfeit_Weight"]=np.where(pd.isnull(cf_test["Counterfeit_Weight"]),14,cf_test['Counterfeit_Weight'])

In [175]:
cf_train.isnull().sum()

Counterfeit_Weight     0
Active_Since           0
Medicine_MRP           0
Medicine_Type          0
SidEffect_Level        0
Availability_rating    0
Area_Type              0
Area_City_Type         0
Area_dist_level        0
Counterfeit_Sales      0
dtype: int64

In [176]:
for col in cf_train.select_dtypes(['object']).columns:
    print(col,':',cf_train[col].nunique())

Medicine_Type : 16
SidEffect_Level : 2
Area_Type : 4
Area_City_Type : 3
Area_dist_level : 4


In [177]:
for col in ['Medicine_Type','SidEffect_Level','Area_Type','Area_City_Type','Area_dist_level']:
    
    temp=pd.get_dummies(cf_train[col],prefix=col,drop_first=True)
    cf_train=pd.concat([temp,cf_train],1)
    cf_train.drop([col],1,inplace=True)


In [178]:
for col in ['Medicine_Type','SidEffect_Level','Area_Type','Area_City_Type','Area_dist_level']:
    
    temp=pd.get_dummies(cf_test[col],prefix=col,drop_first=True)
    cf_test=pd.concat([temp,cf_test],1)
    cf_test.drop([col],1,inplace=True)


In [179]:
cf_train.dtypes

Area_dist_level_Medium                uint8
Area_dist_level_Small                 uint8
Area_dist_level_Unknown               uint8
Area_City_Type_Tier 2                 uint8
Area_City_Type_Tier 3                 uint8
Area_Type_DownTown                    uint8
Area_Type_Industrial                  uint8
Area_Type_MidTownResidential          uint8
SidEffect_Level_mild                  uint8
Medicine_Type_Antacids                uint8
Medicine_Type_Antibiotics             uint8
Medicine_Type_Antifungal              uint8
Medicine_Type_Antimalarial            uint8
Medicine_Type_Antipyretics            uint8
Medicine_Type_Antiseptics             uint8
Medicine_Type_Antiviral               uint8
Medicine_Type_Cardiac                 uint8
Medicine_Type_Hreplacements           uint8
Medicine_Type_Mstablizers             uint8
Medicine_Type_MuscleRelaxants         uint8
Medicine_Type_OralContraceptives      uint8
Medicine_Type_Statins                 uint8
Medicine_Type_Stimulants        

In [180]:
for col in ['Active_Since']:
    
    temp=pd.get_dummies(cf_train[col],prefix=col,drop_first=True)
    cf_train=pd.concat([temp,cf_train],1)
    cf_train.drop([col],1,inplace=True)

In [181]:
for col in ['Active_Since']:
    
    temp=pd.get_dummies(cf_test[col],prefix=col,drop_first=True)
    cf_test=pd.concat([temp,cf_test],1)
    cf_test.drop([col],1,inplace=True)

In [182]:
cf1_train,cf1_test=train_test_split(cf_train,test_size=0.2,random_state=2)

In [183]:
cf_train.dtypes

Active_Since_1985                     uint8
Active_Since_1995                     uint8
Active_Since_1996                     uint8
Active_Since_1997                     uint8
Active_Since_2000                     uint8
Active_Since_2002                     uint8
Active_Since_2005                     uint8
Active_Since_2007                     uint8
Area_dist_level_Medium                uint8
Area_dist_level_Small                 uint8
Area_dist_level_Unknown               uint8
Area_City_Type_Tier 2                 uint8
Area_City_Type_Tier 3                 uint8
Area_Type_DownTown                    uint8
Area_Type_Industrial                  uint8
Area_Type_MidTownResidential          uint8
SidEffect_Level_mild                  uint8
Medicine_Type_Antacids                uint8
Medicine_Type_Antibiotics             uint8
Medicine_Type_Antifungal              uint8
Medicine_Type_Antimalarial            uint8
Medicine_Type_Antipyretics            uint8
Medicine_Type_Antiseptics       

In [184]:
cf_train["Counterfeit_Weight"]=pd.to_numeric(cf_train["Counterfeit_Weight"],errors="coerce")

In [185]:
cf_test["Counterfeit_Weight"]=pd.to_numeric(cf_test["Counterfeit_Weight"],errors="coerce")

In [186]:
cf_train.dtypes

Active_Since_1985                     uint8
Active_Since_1995                     uint8
Active_Since_1996                     uint8
Active_Since_1997                     uint8
Active_Since_2000                     uint8
Active_Since_2002                     uint8
Active_Since_2005                     uint8
Active_Since_2007                     uint8
Area_dist_level_Medium                uint8
Area_dist_level_Small                 uint8
Area_dist_level_Unknown               uint8
Area_City_Type_Tier 2                 uint8
Area_City_Type_Tier 3                 uint8
Area_Type_DownTown                    uint8
Area_Type_Industrial                  uint8
Area_Type_MidTownResidential          uint8
SidEffect_Level_mild                  uint8
Medicine_Type_Antacids                uint8
Medicine_Type_Antibiotics             uint8
Medicine_Type_Antifungal              uint8
Medicine_Type_Antimalarial            uint8
Medicine_Type_Antipyretics            uint8
Medicine_Type_Antiseptics       

In [187]:
ld_train, ld_test = train_test_split(cf_train, test_size = 0.2,random_state=2)

In [188]:
import pandas as pd
import math
from sklearn.cross_validation import train_test_split
from sklearn import tree
import numpy as np
from sklearn.cross_validation import KFold
import matplotlib.pyplot as plt


In [189]:
x_train=ld_train.drop(["Counterfeit_Sales"],1)
y_train=ld_train["Counterfeit_Sales"]
x_test=ld_test.drop(["Counterfeit_Sales"],1)
y_test=ld_test["Counterfeit_Sales"]

In [190]:
x_train.reset_index(drop=True,inplace=True)
y_train.reset_index(drop=True,inplace=True)

In [191]:
from sklearn.grid_search import GridSearchCV


In [192]:
import time
start_time = time.time()

param_grid1 = {'max_depth':list(range(2,81,50)),
               'max_features':list(range(3,16,10)),
               "max_leaf_nodes":list(range(10,50,35)),
                'min_impurity_decrease':list(range(1,50,100)),
               'presort':[True,False],
               'random_state':[2,3,4,5,6,7],
               
               'min_samples_split':[2,3,4,5]}
grid = GridSearchCV(tree.DecisionTreeRegressor(criterion="mse",random_state=2),param_grid=param_grid1,cv=10)
grid.fit(x_train,y_train)


print("--- %s seconds ---" % (time.time() - start_time))

--- 61.983086347579956 seconds ---


Line number 1,2 and 10 have nothing to do with run of your algorithm , we are putting it to time the runtime. For me as you can see it took around ~1 minutes. It will vary for you depending upon your RAM, and backgroun processes running when you are executing this on your machine.

In [193]:
print(grid.best_estimator_)

DecisionTreeRegressor(criterion='mse', max_depth=52, max_features=13,
           max_leaf_nodes=45, min_impurity_decrease=1,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=2, splitter='best')


We'll use this , in our final model as given below

In [194]:
dtree=tree.DecisionTreeRegressor(criterion='mse', max_depth=52, max_features=13,
           max_leaf_nodes=45, min_impurity_decrease=1,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0, presort=True,
           random_state=2, splitter='best')

In [195]:
dtree.fit(x_train,y_train)

DecisionTreeRegressor(criterion='mse', max_depth=52, max_features=13,
           max_leaf_nodes=45, min_impurity_decrease=1,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0, presort=True,
           random_state=2, splitter='best')

In [196]:
predicted=dtree.predict(x_test)

In [197]:
residual=predicted-y_test

In [198]:
rmse_dtree=np.sqrt(np.dot(residual,residual)/len(predicted))

rmse_dtree

1155.1747729458507

In [199]:
from sklearn.metrics import mean_absolute_error
%matplotlib inline

mae=mean_absolute_error(y_test,predicted)

In [200]:
mae

802.9225356915717

In [201]:
score=1-(mae)/1160
score

0.30782540026588645

In [202]:
p=dtree.predict(cf_test)

In [203]:
p

array([1036.4690835 , 3774.39823768, 1426.85046474, ..., 3069.59803187,
       3623.49708   , 3774.39823768])

In [204]:
submission=pd.DataFrame(list(p),
                       columns=['Counterfeit_Sales'])

In [205]:
submission.to_csv('submission_final.csv',index=False)